In [11]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from collections import Counter
import warnings
# if you keras is not using tensorflow as backend set "KERAS_BACKEND=tensorflow" use this command
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils 
from keras.datasets import mnist 
import seaborn as sns
from keras.initializers import RandomNormal

from sklearn.model_selection import train_test_split 

# Regression Example With Boston Dataset: Standardized and Larger
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error

from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from keras.models import Sequential
# from tensorflow.keras.models import Dense,Dropout
from tensorflow.keras import layers
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.normalization import BatchNormalization
import pickle
from tqdm import tqdm
import os

In [3]:
data = read_csv('C:\\Users\\rahul\\Desktop\\E-comm sale\\Dataset\\Train.csv',encoding='utf-8')

mark_sqrt = np.sqrt(data['Market_Category'])
grade_sqrt = np.sqrt(data['Grade'])
lcp_sqrt = np.sqrt(data['Low_Cap_Price'])
hcp_sqrt = np.sqrt(data['High_Cap_Price'])
noise = np.random.uniform(0,1,len(data))
demand_log = np.log((data['Demand']+noise))
Corrected_data = data
Corrected_data['Market_Category'] = mark_sqrt
Corrected_data['Grade'] = grade_sqrt
Corrected_data['Demand'] = demand_log
Corrected_data['Low_Cap_Price'] = lcp_sqrt
Corrected_data['High_Cap_Price'] = hcp_sqrt

print(Corrected_data.head())

  Item_Id        Date  State_of_Country  Market_Category  Product_Category  \
0    IT_1  2007-07-05                 0         0.000000                 0   
1    IT_2  2007-07-05                 0         1.000000                 0   
2    IT_3  2007-07-05                 0        10.148892                 0   
3    IT_4  2007-07-05                 0        10.148892                 0   
4    IT_5  2007-07-05                 0        10.148892                 0   

      Grade    Demand  Low_Cap_Price  High_Cap_Price  
0  0.000000  0.185890      52.773099       65.115282  
1  0.000000  0.148331      59.782941       68.963759  
2  1.000000  0.531851      77.317527       98.331073  
3  0.000000 -2.442184      75.372409       91.175655  
4  1.414214 -2.492985      70.171219       85.188027  


In [4]:
#Performing the standard train test split of 80-20
y_caps = Corrected_data['Low_Cap_Price'].values
print(y_caps)
Corrected_data = Corrected_data.drop(['Low_Cap_Price'],axis=1)
print(Corrected_data.loc[:,'State_of_Country':'High_Cap_Price'].head())

[52.77309921 59.78294071 77.31752712 ... 68.70953355 71.28814768
 62.44197306]
   State_of_Country  Market_Category  Product_Category     Grade    Demand  \
0                 0         0.000000                 0  0.000000  0.185890   
1                 0         1.000000                 0  0.000000  0.148331   
2                 0        10.148892                 0  1.000000  0.531851   
3                 0        10.148892                 0  0.000000 -2.442184   
4                 0        10.148892                 0  1.414214 -2.492985   

   High_Cap_Price  
0       65.115282  
1       68.963759  
2       98.331073  
3       91.175655  
4       85.188027  


In [5]:
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
X_train, X_test, y_train, y_test = train_test_split(Corrected_data.loc[:,'State_of_Country':'High_Cap_Price'], y_caps, test_size=0.1)
print(len(X_train))
print(len(X_test))

8818
980


## Try various versions architectures of MLP and check if they could outperform regression models ##

In [6]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras
#lets add Batch normalization only in second and third layer

from keras.layers import Dropout

model_drop = Sequential()

model_drop.add(Dense(50, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(20, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(10, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(1, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.compile(loss='mean_squared_error', optimizer='adam')
model_drop.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                350       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)               

In [40]:
epochs = 1000
batch_size = 64

history = model_drop.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch_size,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model_drop.evaluate(X_test, y_test, verbose=0)
print("MSE: %.3f%%" % (scores*100))


Train on 8818 samples, validate on 980 samples
Epoch 1/1000
8818/8818 [==============================] - 0s 32us/step - loss: 260.7495 - val_loss: 483.3140
Epoch 2/1000
8818/8818 [==============================] - 0s 31us/step - loss: 276.1506 - val_loss: 435.7494
Epoch 3/1000
8818/8818 [==============================] - 0s 31us/step - loss: 255.8596 - val_loss: 487.2972
Epoch 4/1000
8818/8818 [==============================] - 0s 31us/step - loss: 251.2828 - val_loss: 483.3004
Epoch 5/1000
8818/8818 [==============================] - 0s 31us/step - loss: 252.3895 - val_loss: 499.0031
Epoch 6/1000
8818/8818 [==============================] - 0s 30us/step - loss: 247.4198 - val_loss: 474.8879
Epoch 7/1000
8818/8818 [==============================] - 0s 32us/step - loss: 249.0109 - val_loss: 451.7538
Epoch 8/1000
8818/8818 [==============================] - 0s 31us/step - loss: 246.2726 - val_loss: 487.2391
Epoch 9/1000
8818/8818 [==============================] - 0s 30us/step - loss: 24

In [41]:
y_pred = model_drop.predict(X_test)
mean_sq_log_error = mean_squared_log_error(y_pred, y_test)
print("The best mean sq log error for our MLP regression model is: ",mean_sq_log_error)

The best mean sq log error for our MLP regression model is:  0.07090473908867129


In [7]:
#C:\Users\rahul\Desktop\Novartis Challenge\Dataset
test_data = pd.read_csv('Test.csv',encoding='utf-8')

#Correcting skew in test data
grade_sqrt = np.sqrt(test_data['Grade'])
mark_sqrt = np.sqrt(test_data['Market_Category'])
noise = np.random.uniform(0,1,len(test_data))
demand_log = np.log((test_data['Demand']+noise))
hcp_sqrt = np.sqrt(test_data['High_Cap_Price'])

In [8]:
print(test_data.head())

     Item_Id        Date  State_of_Country  ...  Grade  Demand  High_Cap_Price
0  IT_265079  2014-01-19                 0  ...      0     0.1            5303
1  IT_265087  2014-01-19                 0  ...      2     1.4           10492
2  IT_265107  2014-01-19                 0  ...      0    13.4           24706
3  IT_265139  2014-01-19                17  ...      3     5.5           23464
4  IT_265142  2014-01-19                18  ...      2     0.0            6222

[5 rows x 8 columns]


In [9]:
corr_test_data = test_data.loc[:,'State_of_Country':'High_Cap_Price']
corr_test_data['Market_Category'] = mark_sqrt
corr_test_data['Grade'] = grade_sqrt
corr_test_data['Demand'] = demand_log
corr_test_data['High_Cap_Price'] = hcp_sqrt

print(corr_test_data.head())


   State_of_Country  Market_Category  ...    Demand  High_Cap_Price
0                 0         1.732051  ... -0.636213       72.821700
1                 0        16.370706  ...  0.592652      102.430464
2                 0        17.888544  ...  2.655280      157.181424
3                17        18.920888  ...  1.769081      153.179633
4                18         4.795832  ... -2.017054       78.879655

[5 rows x 6 columns]


In [46]:
X_Test = corr_test_data
Y_Pred = model_drop.predict(X_Test)

#Take squares of values as the predicted values of square roots of LCP
Y_Pred = Y_Pred ** 2
# Y_Pred = Y_Pred.tolist()

results = test_data[['Item_Id']]
print(type(results))

results['Low_Cap_Price'] = Y_Pred 
#How our final submission files look like!
print(results.head())
print(results.tail())
results.to_csv("Results_MLP.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
     Item_Id  Low_Cap_Price
0  IT_265079    2703.678711
1  IT_265087    2854.100586
2  IT_265107    3317.406494
3  IT_265139    4414.772949
4  IT_265142    2854.100586
        Item_Id  Low_Cap_Price
5758  IT_361691    3209.871826
5759  IT_361694    2913.323975
5760  IT_361699    3013.326172
5761  IT_361719    2854.100586
5762  IT_361723    3313.190186


In [47]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras
#lets add Batch normalization only in second and third layer

from keras.layers import Dropout

model_drop = Sequential()

model_drop.add(Dense(100, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(40, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(20, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(1, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.compile(loss='mean_squared_error', optimizer='adam')
model_drop.summary()


Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_86 (Dense)             (None, 100)               700       
_________________________________________________________________
dropout_82 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 40)                4040      
_________________________________________________________________
dropout_83 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_88 (Dense)             (None, 20)                820       
_________________________________________________________________
batch_normalization_57 (Batc (None, 20)                80        
_________________________________________________________________
dropout_84 (Dropout)         (None, 20)              

In [53]:
epochs = 200
batch_size = 64

history = model_drop.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch_size,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model_drop.evaluate(X_test, y_test, verbose=0)
print("MSE: %.3f%%" % (scores*100))


Train on 8818 samples, validate on 980 samples
Epoch 1/200
8818/8818 [==============================] - 0s 37us/step - loss: 133.0247 - val_loss: 140.8727
Epoch 2/200
8818/8818 [==============================] - 0s 32us/step - loss: 134.5899 - val_loss: 137.4042
Epoch 3/200
8818/8818 [==============================] - 0s 32us/step - loss: 129.4318 - val_loss: 158.5668
Epoch 4/200
8818/8818 [==============================] - 0s 33us/step - loss: 135.1095 - val_loss: 134.5542
Epoch 5/200
8818/8818 [==============================] - 0s 33us/step - loss: 134.4310 - val_loss: 142.5567
Epoch 6/200
8818/8818 [==============================] - 0s 32us/step - loss: 138.8710 - val_loss: 153.4385
Epoch 7/200
8818/8818 [==============================] - 0s 32us/step - loss: 138.5568 - val_loss: 138.2309
Epoch 8/200
8818/8818 [==============================] - 0s 33us/step - loss: 134.1878 - val_loss: 134.0553
Epoch 9/200
8818/8818 [==============================] - 0s 33us/step - loss: 134.8449 - 

In [54]:
y_pred = model_drop.predict(X_test)
mean_sq_log_error = mean_squared_log_error(y_pred, y_test)
print("The best mean sq log error for our MLP regression model is: ",mean_sq_log_error)

The best mean sq log error for our MLP regression model is:  0.043057507728599474


In [52]:
X_Test = corr_test_data
Y_Pred = model_drop.predict(X_Test)

#Take squares of values as the predicted values of square roots of LCP
Y_Pred = Y_Pred ** 2
# Y_Pred = Y_Pred.tolist()

results = test_data[['Item_Id']]
print(type(results))

results['Low_Cap_Price'] = Y_Pred 
#How our final submission files look like!
print(results.head())
print(results.tail())
results.to_csv("Results_MLP2.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
     Item_Id  Low_Cap_Price
0  IT_265079    3003.275391
1  IT_265087    3077.187256
2  IT_265107    3277.520996
3  IT_265139    5344.744141
4  IT_265142    2632.651611
        Item_Id  Low_Cap_Price
5758  IT_361691    4716.280273
5759  IT_361694    4505.513672
5760  IT_361699    4578.684570
5761  IT_361719    4361.750488
5762  IT_361723    4793.246582


In [13]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras
#lets add Batch normalization only in second and third layer
from tensorflow.keras.models import Sequential
# from tensorflow.keras.models import Dropout

model_drop = Sequential()

model_drop.add(layers.Dense(5, activation='relu', input_dim=6, kernel_initializer='normal'))

model_drop.add(layers.Dense(10, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(layers.Dropout(0.25))

model_drop.add(layers.Dense(1, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(layers.BatchNormalization())
model_drop.compile(loss='mean_squared_error', optimizer='adam')
model_drop.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5)                 35        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                60        
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
_________________________________________________________________
batch_normalization (BatchNo (None, 1)                 4         
Total params: 110
Trainable params: 108
Non-trainable params: 2
_________________________________________________________________


In [15]:
epochs = 750
batch_size = 64

history = model_drop.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model_drop.evaluate(X_test, y_test, verbose=0)
print("MSE: %.3f%%" % (scores*100))


Epoch 1/750
138/138 [==============================] - 1s 6ms/step - loss: 3477.2764 - val_loss: 3445.5854
Epoch 2/750
138/138 [==============================] - 0s 2ms/step - loss: 3452.4451 - val_loss: 3445.1282
Epoch 3/750
138/138 [==============================] - 0s 2ms/step - loss: 3430.3416 - val_loss: 3392.3838
Epoch 4/750
138/138 [==============================] - 0s 2ms/step - loss: 3409.5554 - val_loss: 3356.7412
Epoch 5/750
138/138 [==============================] - 0s 2ms/step - loss: 3388.9641 - val_loss: 3300.5669
Epoch 6/750
138/138 [==============================] - 0s 2ms/step - loss: 3369.8228 - val_loss: 3331.4998
Epoch 7/750
138/138 [==============================] - 0s 2ms/step - loss: 3350.5059 - val_loss: 3256.6675
Epoch 8/750
138/138 [==============================] - 0s 2ms/step - loss: 3331.6501 - val_loss: 3268.4734
Epoch 9/750
138/138 [==============================] - 0s 2ms/step - loss: 3312.5554 - val_loss: 3202.7346
Epoch 10/750
138/138 [===============

Epoch 77/750
138/138 [==============================] - 0s 2ms/step - loss: 2241.0325 - val_loss: 2060.1143
Epoch 78/750
138/138 [==============================] - 0s 2ms/step - loss: 2226.7781 - val_loss: 2232.1001
Epoch 79/750
138/138 [==============================] - 0s 2ms/step - loss: 2214.8420 - val_loss: 2347.6196
Epoch 80/750
138/138 [==============================] - 0s 2ms/step - loss: 2200.2820 - val_loss: 2166.7483
Epoch 81/750
138/138 [==============================] - 0s 2ms/step - loss: 2188.0061 - val_loss: 2174.1741
Epoch 82/750
138/138 [==============================] - 0s 2ms/step - loss: 2173.6846 - val_loss: 2034.3400
Epoch 83/750
138/138 [==============================] - 0s 2ms/step - loss: 2162.5229 - val_loss: 2000.1440
Epoch 84/750
138/138 [==============================] - 0s 2ms/step - loss: 2147.9922 - val_loss: 1970.1249
Epoch 85/750
138/138 [==============================] - 0s 2ms/step - loss: 2135.1709 - val_loss: 2061.2061
Epoch 86/750
138/138 [======

138/138 [==============================] - 0s 2ms/step - loss: 1397.8921 - val_loss: 1443.6752
Epoch 153/750
138/138 [==============================] - 0s 2ms/step - loss: 1388.4106 - val_loss: 1505.7972
Epoch 154/750
138/138 [==============================] - 0s 2ms/step - loss: 1377.2919 - val_loss: 1284.7947
Epoch 155/750
138/138 [==============================] - 0s 2ms/step - loss: 1370.3617 - val_loss: 1148.6471
Epoch 156/750
138/138 [==============================] - 0s 3ms/step - loss: 1359.3671 - val_loss: 1363.0248
Epoch 157/750
138/138 [==============================] - 0s 3ms/step - loss: 1350.7850 - val_loss: 1435.6282
Epoch 158/750
138/138 [==============================] - 0s 3ms/step - loss: 1338.7808 - val_loss: 1026.3160
Epoch 159/750
138/138 [==============================] - 0s 4ms/step - loss: 1331.0682 - val_loss: 1120.9708
Epoch 160/750
138/138 [==============================] - 1s 4ms/step - loss: 1321.3148 - val_loss: 1256.8391
Epoch 161/750
138/138 [==========

138/138 [==============================] - 1s 4ms/step - loss: 780.1246 - val_loss: 662.1802
Epoch 228/750
138/138 [==============================] - 1s 4ms/step - loss: 773.5773 - val_loss: 599.4601
Epoch 229/750
138/138 [==============================] - 0s 3ms/step - loss: 766.7607 - val_loss: 652.2410
Epoch 230/750
138/138 [==============================] - 0s 3ms/step - loss: 759.7089 - val_loss: 703.9617
Epoch 231/750
138/138 [==============================] - 0s 2ms/step - loss: 751.3770 - val_loss: 689.1073
Epoch 232/750
138/138 [==============================] - 0s 2ms/step - loss: 747.4464 - val_loss: 550.8272
Epoch 233/750
138/138 [==============================] - 0s 2ms/step - loss: 739.0406 - val_loss: 609.5779
Epoch 234/750
138/138 [==============================] - 0s 3ms/step - loss: 734.2056 - val_loss: 670.0451
Epoch 235/750
138/138 [==============================] - 0s 2ms/step - loss: 727.7924 - val_loss: 629.4531
Epoch 236/750
138/138 [============================

138/138 [==============================] - 0s 3ms/step - loss: 364.3246 - val_loss: 286.3207
Epoch 304/750
138/138 [==============================] - 1s 5ms/step - loss: 358.9239 - val_loss: 331.2787
Epoch 305/750
138/138 [==============================] - 1s 4ms/step - loss: 356.5281 - val_loss: 276.8284
Epoch 306/750
138/138 [==============================] - 0s 2ms/step - loss: 352.3824 - val_loss: 439.3549
Epoch 307/750
138/138 [==============================] - 0s 3ms/step - loss: 349.9741 - val_loss: 307.1996
Epoch 308/750
138/138 [==============================] - 0s 2ms/step - loss: 343.8459 - val_loss: 273.9545
Epoch 309/750
138/138 [==============================] - 0s 3ms/step - loss: 340.9509 - val_loss: 276.2709
Epoch 310/750
138/138 [==============================] - 0s 2ms/step - loss: 336.7177 - val_loss: 335.4845
Epoch 311/750
138/138 [==============================] - 0s 2ms/step - loss: 334.5771 - val_loss: 311.2572
Epoch 312/750
138/138 [============================

138/138 [==============================] - 0s 2ms/step - loss: 154.2540 - val_loss: 125.5338
Epoch 380/750
138/138 [==============================] - 0s 2ms/step - loss: 153.6185 - val_loss: 105.8655
Epoch 381/750
138/138 [==============================] - 0s 2ms/step - loss: 149.9199 - val_loss: 113.0416
Epoch 382/750
138/138 [==============================] - 0s 2ms/step - loss: 151.0103 - val_loss: 109.3260
Epoch 383/750
138/138 [==============================] - 0s 2ms/step - loss: 145.8387 - val_loss: 112.0795
Epoch 384/750
138/138 [==============================] - 0s 2ms/step - loss: 145.7449 - val_loss: 127.4820
Epoch 385/750
138/138 [==============================] - 0s 3ms/step - loss: 145.5175 - val_loss: 118.4939
Epoch 386/750
138/138 [==============================] - 1s 4ms/step - loss: 145.8200 - val_loss: 111.8183
Epoch 387/750
138/138 [==============================] - 0s 3ms/step - loss: 144.3668 - val_loss: 134.8456
Epoch 388/750
138/138 [============================

138/138 [==============================] - 0s 2ms/step - loss: 115.4853 - val_loss: 90.4144
Epoch 457/750
138/138 [==============================] - 0s 2ms/step - loss: 114.1407 - val_loss: 100.5334
Epoch 458/750
138/138 [==============================] - 0s 2ms/step - loss: 112.1404 - val_loss: 94.6166
Epoch 459/750
138/138 [==============================] - 0s 2ms/step - loss: 115.4003 - val_loss: 93.6303
Epoch 460/750
138/138 [==============================] - 0s 3ms/step - loss: 114.4200 - val_loss: 102.1816
Epoch 461/750
138/138 [==============================] - 1s 5ms/step - loss: 115.3136 - val_loss: 91.1604
Epoch 462/750
138/138 [==============================] - 1s 4ms/step - loss: 112.3498 - val_loss: 90.5065
Epoch 463/750
138/138 [==============================] - 0s 3ms/step - loss: 115.5905 - val_loss: 92.0253
Epoch 464/750
138/138 [==============================] - 0s 2ms/step - loss: 115.0633 - val_loss: 106.2354
Epoch 465/750
138/138 [==============================] - 

138/138 [==============================] - 0s 2ms/step - loss: 113.4424 - val_loss: 90.7205
Epoch 534/750
138/138 [==============================] - 0s 2ms/step - loss: 114.0814 - val_loss: 93.2498
Epoch 535/750
138/138 [==============================] - 0s 3ms/step - loss: 114.6226 - val_loss: 98.3969
Epoch 536/750
138/138 [==============================] - 1s 4ms/step - loss: 114.8141 - val_loss: 103.1880
Epoch 537/750
138/138 [==============================] - 0s 4ms/step - loss: 116.7625 - val_loss: 90.3589
Epoch 538/750
138/138 [==============================] - 0s 3ms/step - loss: 115.1045 - val_loss: 90.9235
Epoch 539/750
138/138 [==============================] - 0s 3ms/step - loss: 113.6705 - val_loss: 91.5154
Epoch 540/750
138/138 [==============================] - 0s 2ms/step - loss: 114.1292 - val_loss: 91.7720
Epoch 541/750
138/138 [==============================] - 0s 3ms/step - loss: 112.9610 - val_loss: 89.2180
Epoch 542/750
138/138 [==============================] - 0s

138/138 [==============================] - 0s 3ms/step - loss: 113.2231 - val_loss: 93.0239
Epoch 611/750
138/138 [==============================] - 0s 3ms/step - loss: 112.4763 - val_loss: 89.9689
Epoch 612/750
138/138 [==============================] - 0s 2ms/step - loss: 113.6289 - val_loss: 93.1182
Epoch 613/750
138/138 [==============================] - 0s 3ms/step - loss: 113.7959 - val_loss: 90.5923
Epoch 614/750
138/138 [==============================] - 1s 5ms/step - loss: 114.6005 - val_loss: 90.3566
Epoch 615/750
138/138 [==============================] - 1s 5ms/step - loss: 114.3819 - val_loss: 93.0664
Epoch 616/750
138/138 [==============================] - 1s 4ms/step - loss: 114.0548 - val_loss: 91.2979
Epoch 617/750
138/138 [==============================] - 0s 3ms/step - loss: 114.2521 - val_loss: 88.8925
Epoch 618/750
138/138 [==============================] - 0s 3ms/step - loss: 112.9896 - val_loss: 98.6793
Epoch 619/750
138/138 [==============================] - 0s 

138/138 [==============================] - 0s 2ms/step - loss: 113.0767 - val_loss: 93.4457
Epoch 688/750
138/138 [==============================] - 0s 3ms/step - loss: 112.2282 - val_loss: 91.0712
Epoch 689/750
138/138 [==============================] - 0s 3ms/step - loss: 110.5568 - val_loss: 90.9297
Epoch 690/750
138/138 [==============================] - 0s 3ms/step - loss: 112.5118 - val_loss: 92.4590
Epoch 691/750
138/138 [==============================] - 0s 3ms/step - loss: 114.6008 - val_loss: 90.9551
Epoch 692/750
138/138 [==============================] - 0s 3ms/step - loss: 113.8333 - val_loss: 91.2097
Epoch 693/750
138/138 [==============================] - 0s 2ms/step - loss: 113.3736 - val_loss: 89.5627
Epoch 694/750
138/138 [==============================] - 0s 2ms/step - loss: 110.7525 - val_loss: 89.4251
Epoch 695/750
138/138 [==============================] - 0s 2ms/step - loss: 114.7924 - val_loss: 95.5967
Epoch 696/750
138/138 [==============================] - 0s 

In [16]:
X_Test = corr_test_data
Y_Pred = model_drop.predict(X_Test)

#Take squares of values as the predicted values of square roots of LCP
Y_Pred = Y_Pred ** 2
# Y_Pred = Y_Pred.tolist()

results = test_data[['Item_Id']]
print(type(results))

results['Low_Cap_Price'] = Y_Pred 
#How our final submission files look like!
print(results.head())
print(results.tail())
results.to_csv("Results_MLP7.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
     Item_Id  Low_Cap_Price
0  IT_265079    3154.678223
1  IT_265087    3091.074707
2  IT_265107    3497.126221
3  IT_265139    7656.860352
4  IT_265142    2636.739258
        Item_Id  Low_Cap_Price
5758  IT_361691    6891.769043
5759  IT_361694    5522.692383
5760  IT_361699    5568.499023
5761  IT_361719    5069.729492
5762  IT_361723    7330.903809


In [79]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras
#lets add Batch normalization only in second and third layer

from keras.layers import Dropout

model_drop = Sequential()

model_drop.add(Dense(300, activation='relu', input_dim=6, kernel_initializer='normal'))


model_drop.add(Dense(150, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(75, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))

model_drop.add(Dense(50, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))
model_drop.add(BatchNormalization())

model_drop.add(Dense(10, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(Dropout(0.5))
model_drop.add(BatchNormalization())


model_drop.add(Dense(1, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.compile(loss='mean_squared_error', optimizer='adam')
model_drop.summary()


Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_109 (Dense)            (None, 300)               2100      
_________________________________________________________________
dense_110 (Dense)            (None, 150)               45150     
_________________________________________________________________
dropout_93 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 75)                11325     
_________________________________________________________________
dropout_94 (Dropout)         (None, 75)                0         
_________________________________________________________________
dense_112 (Dense)            (None, 50)                3800      
_________________________________________________________________
dropout_95 (Dropout)         (None, 50)              

In [16]:
epochs = 250
batch_size = 64

history = model_drop.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch_size,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model_drop.evaluate(X_test, y_test, verbose=0)
print("MSE: %.3f%%" % (scores*100))


TypeError: fit() got an unexpected keyword argument 'nb_epoch'

In [68]:
X_Test = corr_test_data
Y_Pred = model_drop.predict(X_Test)

#Take squares of values as the predicted values of square roots of LCP
Y_Pred = Y_Pred ** 2
# Y_Pred = Y_Pred.tolist()

results = test_data[['Item_Id']]
print(type(results))

results['Low_Cap_Price'] = Y_Pred 
#How our final submission files look like!
print(results.head())
print(results.tail())
results.to_csv("Results_MLP7.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
     Item_Id  Low_Cap_Price
0  IT_265079    3171.403076
1  IT_265087    2649.415527
2  IT_265107    5039.837402
3  IT_265139    7039.211426
4  IT_265142    3029.222900
        Item_Id  Low_Cap_Price
5758  IT_361691    7374.371582
5759  IT_361694    5702.576172
5760  IT_361699    6031.023926
5761  IT_361719    4890.644531
5762  IT_361723    7634.889160


In [17]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras
#lets add Batch normalization only in second and third layer

from keras.layers import Dropout

model_drop = Sequential()

model_drop.add(Dense(20, activation='relu', input_dim=6, kernel_initializer='normal'))

model_drop.add(Dense(10, activation='relu', input_dim=6, kernel_initializer='normal'))

model_drop.add(Dense(1, activation='relu', input_dim=6, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.compile(loss='mean_squared_error', optimizer='adam')
model_drop.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_9 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
_________________________________________________________________
batch_normalization_5 (Batch (None, 1)                 4         
Total params: 365
Trainable params: 363
Non-trainable params: 2
_________________________________________________________________


In [19]:
epochs = 50
batch_size = 64

history = model_drop.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch_size,validation_data=(X_test,y_test))
# Final evaluation of the model
scores = model_drop.evaluate(X_test, y_test, verbose=0)
print("MSE: %.3f%%" % (scores*100))


Train on 8818 samples, validate on 980 samples
Epoch 1/50
8818/8818 [==============================] - 0s 56us/step - loss: 334.4969 - val_loss: 337.1861
Epoch 2/50
8818/8818 [==============================] - 0s 55us/step - loss: 334.4981 - val_loss: 337.1867
Epoch 3/50
8818/8818 [==============================] - 0s 56us/step - loss: 334.4967 - val_loss: 337.1867
Epoch 4/50
8818/8818 [==============================] - 0s 55us/step - loss: 334.4966 - val_loss: 337.1865
Epoch 5/50
8818/8818 [==============================] - 0s 55us/step - loss: 334.4969 - val_loss: 337.1876
Epoch 6/50
8818/8818 [==============================] - 0s 55us/step - loss: 334.4967 - val_loss: 337.1880
Epoch 7/50
8818/8818 [==============================] - 1s 57us/step - loss: 334.4965 - val_loss: 337.1890
Epoch 8/50
8818/8818 [==============================] - 0s 54us/step - loss: 334.4969 - val_loss: 337.1893
Epoch 9/50
8818/8818 [==============================] - 0s 54us/step - loss: 334.4974 - val_loss:

In [ ]:
X_Test = corr_test_data
Y_Pred = model_drop.predict(X_Test)

#Take squares of values as the predicted values of square roots of LCP
Y_Pred = Y_Pred ** 2
# Y_Pred = Y_Pred.tolist()

results = test_data[['Item_Id']]
print(type(results))

results['Low_Cap_Price'] = Y_Pred 
#How our final submission files look like!
print(results.head())
print(results.tail())
results.to_csv("Results_MLP7.csv",index=False)